In [64]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Specify the directory containing your images
directory = 'tomato'

# Load the dataset
dataset = image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='binary',  # Assuming binary classification between healthy and unhealthy
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),  # Adjust the image size as needed
    shuffle=True,
    seed=123,  # For reproducibility
    validation_split=0.2,  # 20% of data used for validation
    subset='training',  # Specify 'training' for training set
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    data_format=None,
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',  # Specify 'validation' for validation set
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    data_format=None,
)

# Preprocess the images to maintain aspect ratio if necessary
def resize_and_pad(image, label):
    image = tf.image.resize_with_pad(image, 256, 256)
    return image, label

dataset = dataset.map(resize_and_pad)
validation_dataset = validation_dataset.map(resize_and_pad)

# Now you can iterate through the dataset to access your images and labels
for images, labels in dataset.take(1):
    print(images.shape, labels.shape)
for images, labels in validation_dataset.take(1):
    print(images.shape, labels.shape)


Found 3500 files belonging to 2 classes.
Using 2800 files for training.
Found 3500 files belonging to 2 classes.
Using 700 files for validation.
(32, 256, 256, 3) (32, 1)
(32, 256, 256, 3) (32, 1)


2024-05-21 17:59:03.751444: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-21 17:59:03.933985: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [65]:
!pip install tensorflow

^C
Traceback (most recent call last):
  File "/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/usr/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/lib/python3.11/site-packages/pip/_internal/build_env.py", line 16, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/usr/lib/python3.11/site-packages/pip/_vendor/packaging/requirements.py", line 10, in <module>
    from pip._vendor.pyparsing import (  # noqa
  File "/usr/lib/python3.11/site-packages/pip/_vendor/pyparsing/__init__.py", line 140, in <module>
 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import numpy as np

# Specify the directory containing your images
directory = 'tomato'

# Load the training dataset
train_dataset = image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',  # Use 'int' for categorical labels
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False
)

# Preprocess the images to maintain aspect ratio if necessary
def resize_and_pad(image, label):
    image = tf.image.resize_with_pad(image, 256, 256)
    return image, label

train_dataset = train_dataset.map(resize_and_pad)
validation_dataset = validation_dataset.map(resize_and_pad)

# Normalize images to the [0, 1] range
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Define the model
num_classes = 2  # Binary classification

model = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(num_classes, activation='softmax'),
])

model.summary()

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10  # Set the number of epochs as needed
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

# Evaluate the model
score = model.evaluate(validation_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Save the model
model.save('model.keras')


Found 3500 files belonging to 2 classes.
Using 2800 files for training.
Found 3500 files belonging to 2 classes.
Using 700 files for validation.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)      │        896 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 128, 128, 32)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 524288)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 16)                │  8,388,624 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 16)                │        272 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 2)                 │         34 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 8,389,826 (32.00 MB)

 Trainable params: 8,389,826 (32.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2024-05-21 16:47:34.290770: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 25165952 bytes after encountering the first element of size 25165952 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-05-21 16:47:34.290982: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-05-21 16:47:34.509891: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-05-21 16:47:34.545342: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-05-21 16:47:34.545470: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-05-21 16:47:34.625462: W external/local_tsl/tsl/framework/cpu_al

88/88 ━━━━━━━━━━━━━━━━━━━━ 70s 758ms/step - accuracy: 0.6366 - loss: 2.4198 - val_accuracy: 0.9814 - val_loss: 0.1249
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 65s 736ms/step - accuracy: 0.9637 - loss: 0.1232 - val_accuracy: 0.9800 - val_loss: 0.0534
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 69s 787ms/step - accuracy: 0.9946 - loss: 0.0352 - val_accuracy: 0.9914 - val_loss: 0.0335
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 76s 862ms/step - accuracy: 0.9956 - loss: 0.0290 - val_accuracy: 0.9843 - val_loss: 0.0532
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 77s 874ms/step - accuracy: 0.9908 - loss: 0.0450 - val_accuracy: 0.9914 - val_loss: 0.0302
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.9972 - loss: 0.0225 - val_accuracy: 0.9914 - val_loss: 0.0253
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 77s 873ms/step - accuracy: 0.9966 - loss: 0.0196 - val_accuracy: 0.9914 - val_loss: 0.0290
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9973 - loss: 0.0100 - val_accuracy: 0.9914 - v

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os

# Load the trained model
model = tf.keras.models.load_model('model.keras')

# Specify the directory containing your new images
new_images_directory = 'new_tomato_images'

# Load the new images dataset
new_images_dataset = image_dataset_from_directory(
    new_images_directory,
    labels=None,  # No labels for new images
    label_mode=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),  # Ensure this matches the input size used during training
    shuffle=False
)

# Preprocess the images (resize and normalize)
def preprocess_image(image):
    image = tf.image.resize_with_pad(image, 256, 256)
    image = tf.cast(image, tf.float32) / 255.0
    return image

new_images_dataset = new_images_dataset.map(preprocess_image)

predictions = model.predict(new_images_dataset)

predicted_classes = np.argmax(predictions, axis=1)
print("Predicted classes:", predicted_classes)




Found 16 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Predicted classes: [0 0 0 0 1 0 1 0 0 1 1 1 1 0 1 0]


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras

# Display
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt


In [3]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()




In [4]:
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions
img_path = "new_tomato_images/0a3f65fc-ef1c-4aed-b235-46bae4e5c0e7___GHLB2 Leaf 9065.JPG"
last_conv_layer_name = "block14_sepconv2_act"
img_size = (256, 256)

In [5]:
img_array = preprocess_input(get_img_array(img_path, size=img_size))

In [6]:
model = tf.keras.models.load_model('model.keras')

In [7]:
model.layers[-1].activation = None

Model output shape: (None, 2)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Received array with shape: (1, 2)